# Testing polynomial fitting methods for approximating gradients

This was a useful tool for comparing predicted gradients from the models against those obtained via a simple local polynomial fit, but results from this notebook were not used in the paper.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import gc
from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KDTree
import pickle


cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
base_dir = os.path.dirname(parent_dir)


src_dir = base_dir + "/src"

sys.path.insert(0, src_dir)

from data_analysis import data_processing, load_data, convert_frames, correct_sensor_bias, plot_quiver_slice
from evaluate import get_poly_gradients_dict


In [ ]:
#########################################
############## Which eMNS? ##############
#########################################

emns = "octomag" # "octomag" or "navion"

if emns == "octomag":
    data_dir = base_dir + "/data/octomag_data/split_dataset"
    data_path = data_dir + "/test_data.pkl"
    store_grads_path = cwd + "/data/poly_grads_test.pkl"
    
elif emns == "navion":
    data_dir = base_dir + "/data/navion_data/split_dataset"
    data_path = data_dir + "/test_data.pkl"
    store_grads_path = cwd + "/data_navion/poly_grads_navion.pkl"    

In [ ]:
data = pd.read_pickle(data_path) 

## Prepare for run

In [ ]:
# Define orders of polynomial fits
orders = [1, 2, 3]

# Define numbers of nearest neighbors to try
nn_values = [3**3, 5**3, 7**3, 9**3, 11**3]

## Run

In [ ]:
grads_dict = get_poly_gradients_dict(data, orders, nn_values)

In [ ]:
grads_dict['gradients'].keys()

## Store

In [ ]:
os.makedirs(os.path.dirname(store_grads_path), exist_ok=True)
with open(store_grads_path, "wb") as f:
    pickle.dump(grads_dict, f)

# Analysis

## Load

In [ ]:
with open(store_grads_path, "rb") as f:
    loaded_grads_dict = pickle.load(f)

## Check divergence and curl

In [ ]:
# ---- compute arrays ----
div_dict = {}       # (order, nn) -> (N,)
curlmag_dict = {}   # (order, nn) -> (N,)

for order in orders:
    for nn in nn_values:
        if order not in loaded_grads_dict['gradients'] or nn not in loaded_grads_dict['gradients'][order]:
            continue

        J = np.asarray(loaded_grads_dict['gradients'][order][nn])  # (N,3,3)

        dBx_dx, dBx_dy, dBx_dz = J[:,0,0], J[:,0,1], J[:,0,2]
        dBy_dx, dBy_dy, dBy_dz = J[:,1,0], J[:,1,1], J[:,1,2]
        dBz_dx, dBz_dy, dBz_dz = J[:,2,0], J[:,2,1], J[:,2,2]

        div = dBx_dx + dBy_dy + dBz_dz
        curl_x = dBz_dy - dBy_dz
        curl_y = dBx_dz - dBz_dx
        curl_z = dBy_dx - dBx_dy
        curl_mag = np.sqrt(curl_x**2 + curl_y**2 + curl_z**2)

        div_dict[(order, nn)] = div
        curlmag_dict[(order, nn)] = curl_mag

# ---- boxplots ----
def boxplot_metric(metric_dict, title, ylabel):
    data, labels = [], []
    for order in orders:
        for nn in nn_values:
            key = (order, nn)
            if key not in metric_dict:
                continue
            data.append(metric_dict[key])
            labels.append(f"{order}_{nn}")

    fig, ax = plt.subplots()
    ax.boxplot(data, labels=labels, showfliers=False)
    ax.set_title(title)
    ax.set_xlabel("order_nn")
    ax.set_ylabel(ylabel)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

boxplot_metric(div_dict, "div distribution by (order, nn)", "div")
boxplot_metric(curlmag_dict, "curl_mag distribution by (order, nn)", "curl_mag")